# Reference
`https://spark.apache.org/docs/latest/streaming-programming-guide.html`

In [ ]:
# Imports
import findspark
findspark.init()
findspark.find()


In [ ]:
from pyspark import SparkConf
conf = SparkConf()
conf.setAppName('JupyterPipeLineTest WINDOWS')
conf.setMaster('spark://spark-master:7077');

In [ ]:
# Setup spark environment
from pyspark import SparkContext
from pyspark.sql.types import *
from pprint import pprint, pformat
sc = SparkContext.getOrCreate(conf)
print('Spark web UI link: ', sc._jsc.sc().uiWebUrl().get())


In [ ]:
from pyspark.streaming import StreamingContext
ssc = StreamingContext(sc, 2)

In [ ]:
lines = ssc.socketTextStream("spark-master", 9999)

In [ ]:
words = lines.flatMap(lambda line: line.split(" "))

# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [ ]:

ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

# go to spark-master and type `nc -lk 9999` to open a data server that we can input text into
# input anything and data will be sent over to this console
# the output put below is working :3

# or just use test server instead